In [73]:
# libraries
import jedi
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling
import pygments
import random 
import re
import seaborn as sns
from scipy import stats as st
import string
from tqdm import tqdm
import this

In [74]:
# Versions
print("jedi version:", jedi.__version__)
print("json version:", json.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("pandas profiling version:", pandas_profiling.__version__)
print("Pygments version:", pygments.__version__)
print("re version:", re.__version__)
print("Seaborn version:", sns.__version__)

jedi version: 0.15.1
json version: 2.0.9
numpy version: 1.17.2
pandas version: 0.25.1
pandas profiling version: 2.4.0
Pygments version: 2.4.2
re version: 2.2.1
Seaborn version: 0.9.0


In [3]:
# Directories & Files
os.listdir()

# Datasets directory
directory = "./datasets/"

## Database Documentation: 

* Equal Opportunities for Women and Men: Promotions, leadership roles, salary increases, incentive programs, etc

* Female Representation in Leadership: Women on the executive team, in senior leadership, etc

* Management Opportunities: Your chances of becoming a manager of teams and talent

* Maternity and Adoptive Leave: Paid parental leave policies, job security, support for returning moms, etc

* Family Growth Support: Access to dedicated lactation rooms, child care, expense reimbursement, etc

* Paid Time Off: Sick days, vacation days, and personal days

* Flexible Work Hours: Ability to set your schedule as long as you get your work done

* Ability to Telecommute: Flexibility to work remotely

* Salary Satisfaction: Salary, merit increases, cost of living adjustments, overall comp

* Sponsorship or Mentorship Program: Official mentorship program, women-focused initiatives or affiliate groups

* Learning Opportunities: On and off-site skills training, speaker series, conferences, etc

* Wellness Initiatives: On-site gym, gym discounts, walking desks, healthy food options, etc

* Employer Responsiveness: Effective channels for elevating issues and concerns

* The People You Work With: Respectful, professional, unbiased, all those good things

* Social Activities and Environment: Happy hours, game room, company outings, and other perks

In [4]:
# creating locations to populate
locations = ['Lisboa', 'Porto', 'Braga', 'Setúbal', 'Aveiro', 'Faro', 'Leiria', 'Coimbra', 
             'Santarém', 'Viseu','Viana do Castelo', 'Madeira', 'Açores', 'Vila Real', 
             'Castelo Branco', 'Évora', 'Guarda', 'Beja', 'Bragança', 'Portalegre']

# creating job_types to populate
job_type = ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Internship', 'Other']

# creating job_levels to populate
job_level = ['Early Career', 'Mid-Level', 'Senior-Level', 'Executive']

# creating job_departments to populate
job_department = ['Accounting & Finance', 'Admin', 'Business Development', 
                  'Business Intelligence', 'Communications', 'Customer Service', 'Design', 
                  'Engineering', 'Human Resources', 'IT', 'Legal', 'Management', 'Marketing', 
                  'Operations', 'Product', 'Production', 'Research & Development', 'Sales', 
                  'Tech', 'Other']

# creating companies, departments and titles to populate
company_department_jobtitle = pd.read_csv(directory + "data_for_random.csv", sep = ";", 
                                     error_bad_lines = False, encoding = 'cp1252')

# creating employment_status
employment_status = ["Current Employee", "Former Employee"]

# creating last_year_of_employment
last_year_of_employment = [i for i in range(2000, 2021)]

In [5]:
# separating into 3 df's
company = company_department_jobtitle["Company"]
department = company_department_jobtitle["Department"]
jobtitle = company_department_jobtitle["Job Title"]

# cleaning the NaN's
company.dropna(inplace = True)
department.dropna(inplace = True)
jobtitle.dropna(inplace = True)

In [6]:
# setting the size of the DF to populate
size = 1000

# setting the distribution to populate ratings
ratings = st.norm.rvs(loc = 3 , scale = 1, size = size)

In [7]:
# setting the distribution to populate salaries
salary = st.norm.rvs(loc = 18000, scale = 8000, size = size)

In [8]:
# creating and filling the columns for the DF
dataset = {"company_id": [i for i in range(size)], 
          "company_name": [random.choice(company) for i in range(size)], 
          "user_id": [1000 + i for i in range(size)], 
          "employment_status": [random.choice(employment_status) for i in range(size)], 
          "last_year_of_employment": [random.choice(last_year_of_employment) for i in range(size)], 
          "location": [random.choice(locations) for i in range(size)], 
          "job_location": [random.choice(locations) for i in range(size)], 
          "job_type": [random.choice(job_type) for i in range(size)], 
          "job_level": [random.choice(job_level) for i in range(size)], 
          "department": [random.choice(job_department) for i in range(size)], 
          "job_title": [random.choice(jobtitle) for i in range(size)], 
          "equal_opportunity": [round(random.choice(ratings)) for i in range(size)], 
          "leadership_female_representation": [round(random.choice(ratings)) for i in range(size)], 
          "women_management_opportunities": [round(random.choice(ratings)) for i in range(size)], 
          "maternity_adoptive_leave": [round(random.choice(ratings)) for i in range(size)], 
          "family_growth_support": [round(random.choice(ratings)) for i in range(size)], 
          "paid_time_off": [round(random.choice(ratings)) for i in range(size)], 
          "flexible_work_hours": [round(random.choice(ratings)) for i in range(size)], 
          "ability_to_telecommute": [round(random.choice(ratings)) for i in range(size)], 
          "salary_satisfaction": [round(random.choice(ratings)) for i in range(size)], 
          "sponsorship_or_mentorship_program": [round(random.choice(ratings)) for i in range(size)], 
          "learning_opportunities": [round(random.choice(ratings)) for i in range(size)], 
          "wellness_initiatives": [round(random.choice(ratings)) for i in range(size)], 
          "employer_responsiveness": [round(random.choice(ratings)) for i in range(size)], 
          "people_you_work_with": [round(random.choice(ratings)) for i in range(size)], 
          "social_activities_environment": [round(random.choice(ratings)) for i in range(size)], 
          "overall_satisfaction": [round(random.choice(ratings)) for i in range(size)], 
          "public_comment": ["".join([random.choice(string.printable) for i in range(15)]) for j in range(size)], 
          "asked_pay_raise_last_12_months": [random.choice([0, 1, 2]) for i in range(size)], 
          "think_pt_buiz_doing_enough_to_address_gender_pay_gap": [random.choice([0, 1]) for i in range(size)],
          "think_gov_doing_enough_to_address_gender_pay_gap": [random.choice([0, 1]) for i in range(size)], 
          "confidence_in_ability_to_negotiate_salary": [round(random.choice(ratings)) for i in range(size)], 
          "sex_orientation": [random.choice(["Heterosexual", "Gay/Lesbian", "Bisexual", "Other"]) for i in range(size)], 
          "gender": [random.choice(["Male", "Female", "Non-Binary", "Other"]) for i in range(size)], 
          "marital_status": [random.choice([0, 1]) for i in range(size)], 
          "children": [random.choice([0, 1]) for i in range(size)], 
          "education": [random.choice([0, 1, 2]) for i in range(size)],
          "salary": [round(random.choice(salary), 2) for i in range(size)]}

In [9]:
# initializing the DF
df = pd.DataFrame(dataset)

In [10]:
# checking the df
df.sample(10)

,company_id,company_name,user_id,employment_status,last_year_of_employment,location,job_location,job_type,job_level,department,...,asked_pay_raise_last_12_months,think_pt_buiz_doing_enough_to_address_gender_pay_gap,think_gov_doing_enough_to_address_gender_pay_gap,confidence_in_ability_to_negotiate_salary,sex_orientation,gender,marital_status,children,education,salary
6,6,Churchill's Port,1006,Current Employee,2003,Madeira,Açores,Temporary,Executive,Research & Development,...,2,1,0,4.0,Bisexual,Non-Binary,0,1,1,20219.74
715,715,NOS,1715,Current Employee,2001,Porto,Faro,Temporary,Early Career,Management,...,0,1,0,3.0,Heterosexual,Male,1,1,0,25941.85
818,818,Renova,1818,Current Employee,2016,Leiria,Évora,Other,Mid-Level,Marketing,...,1,0,0,3.0,Gay/Lesbian,Male,1,1,0,17139.20
712,712,Renova,1712,Current Employee,2013,Braga,Coimbra,Temporary,Executive,Research & Development,...,0,1,1,3.0,Gay/Lesbian,Other,1,1,1,31087.15
90,90,Renova,1090,Former Employee,2007,Beja,Lisboa,Contract,Mid-Level,Communications,...,2,0,1,4.0,Bisexual,Male,0,1,2,26379.19
187,187,Central de Cervejas,1187,Current Employee,2006,Braga,Setúbal,Temporary,Executive,Business Development,...,1,0,1,4.0,Heterosexual,Male,0,1,0,23589.35
467,467,Galp Energia,1467,Current Employee,2018,Lisboa,Évora,Part-time,Senior-Level,Marketing,...,0,0,1,2.0,Gay/Lesbian,Other,1,0,0,14104.52
134,134,Quidgest,1134,Current Employee,2011,Madeira,Açores,Part-time,Senior-Level,Research & Development,...,2,0,0,3.0,Bisexual,Male,0,1,1,28121.71
734,734,Bial,1734,Current Employee,2009,Portalegre,Braga,Other,Early Career,IT,...,2,0,0,3.0,Heterosexual,Other,0,0,0,29607.56
629,629,Porto Editora,1629,Former Employee,2009,Santarém,Coimbra,Full-time,Mid-Level,Human Resources,...,1,0,0,1.0,Other,Male,1,1,2,18815.66


In [11]:
# changing last year of employment to be coherent w/last_year_of_employment
current_employee = df["employment_status"] == "Current Employee"
year = "last_year_of_employment"

df.loc[current_employee, year] = 2020

In [12]:
df.sample(10)

,company_id,company_name,user_id,employment_status,last_year_of_employment,location,job_location,job_type,job_level,department,...,asked_pay_raise_last_12_months,think_pt_buiz_doing_enough_to_address_gender_pay_gap,think_gov_doing_enough_to_address_gender_pay_gap,confidence_in_ability_to_negotiate_salary,sex_orientation,gender,marital_status,children,education,salary
612,612,"CTT Correios de Portugal, S.A. (CTT)",1612,Current Employee,2020,Bragança,Aveiro,Internship,Senior-Level,Marketing,...,0,0,0,2.0,Bisexual,Other,1,1,1,17819.48
557,557,Corticeira Amorim,1557,Current Employee,2020,Bragança,Viseu,Internship,Executive,Operations,...,2,0,1,3.0,Other,Female,1,1,1,17309.68
696,696,Tabaqueira,1696,Former Employee,2017,Coimbra,Braga,Internship,Executive,Legal,...,0,0,0,1.0,Bisexual,Other,0,0,2,25414.55
368,368,Sovena Group,1368,Current Employee,2020,Santarém,Coimbra,Contract,Executive,IT,...,2,1,0,3.0,Heterosexual,Non-Binary,1,0,0,19138.15
758,758,Cofina,1758,Former Employee,2009,Setúbal,Braga,Temporary,Early Career,Engineering,...,0,1,0,5.0,Other,Other,1,0,2,25941.85
932,932,Glintt,1932,Former Employee,2012,Braga,Faro,Other,Early Career,Product,...,1,1,1,4.0,Heterosexual,Female,0,1,2,28554.42
755,755,Impresa,1755,Current Employee,2020,Leiria,Santarém,Contract,Executive,Sales,...,1,1,1,4.0,Other,Female,0,0,0,40898.17
317,317,Biotecnol,1317,Current Employee,2020,Leiria,Faro,Contract,Executive,Other,...,1,0,0,3.0,Bisexual,Male,1,1,1,7240.18
734,734,Bial,1734,Current Employee,2020,Portalegre,Braga,Other,Early Career,IT,...,2,0,0,3.0,Heterosexual,Other,0,0,0,29607.56
395,395,Cimpor,1395,Current Employee,2020,Guarda,Setúbal,Full-time,Senior-Level,Customer Service,...,2,0,0,3.0,Other,Other,0,0,0,16250.89


In [13]:
# reducing the table
df.drop(["user_id", "employment_status", "job_location", "public_comment"], axis = 1, 
        inplace = True)

In [70]:
# avg's of sections
print(df.columns)
df["opportunity"] = round((df["equal_opportunity"] + df["leadership_female_representation"] + df["women_management_opportunities"]) / 3, 2)
df["family"] = round((df["maternity_adoptive_leave"] + df["family_growth_support"]) / 2, 2)
df["schedule_flex"] = round((df["paid_time_off"] + df["flexible_work_hours"] + df["ability_to_telecommute"]) / 3, 2)
df["enrichment"] = round((df["salary_satisfaction"] + df["sponsorship_or_mentorship_program"] + df["learning_opportunities"] + df["wellness_initiatives"]) / 4, 2)
df["culture"] = round((df["employer_responsiveness"] + df["people_you_work_with"] + df["social_activities_environment"]) / 3, 2)

# changing overall rating
df["overall_satisfaction"] = round((df["opportunity"] + df["family"] + df["schedule_flex"] + df["enrichment"] + df["culture"]) / 5, 2)

# checking
df["opportunity"]
df["family"]
df["schedule_flex"]
df["enrichment"]
df["culture"]
df["overall_satisfaction"]
df

Index(['company_id', 'company_name', 'last_year_of_employment', 'location',
       'job_type', 'job_level', 'department', 'job_title', 'equal_opportunity',
       'leadership_female_representation', 'women_management_opportunities',
       'maternity_adoptive_leave', 'family_growth_support', 'paid_time_off',
       'flexible_work_hours', 'ability_to_telecommute', 'salary_satisfaction',
       'sponsorship_or_mentorship_program', 'learning_opportunities',
       'wellness_initiatives', 'employer_responsiveness',
       'people_you_work_with', 'social_activities_environment',
       'overall_satisfaction', 'asked_pay_raise_last_12_months',
       'think_pt_buiz_doing_enough_to_address_gender_pay_gap',
       'think_gov_doing_enough_to_address_gender_pay_gap',
       'confidence_in_ability_to_negotiate_salary', 'sex_orientation',
       'gender', 'marital_status', 'children', 'education', 'salary',
       'opportunity', 'family', 'schedule_flex', 'enrichment', 'culture'],
      dtype='obj

,company_id,company_name,last_year_of_employment,location,job_type,job_level,department,job_title,equal_opportunity,leadership_female_representation,...,gender,marital_status,children,education,salary,opportunity,family,schedule_flex,enrichment,culture
0,0,Jerónimo Martins,2013,Lisboa,Full-time,Executive,Tech,Bookkeeper,3.0,3.0,...,Other,0,0,0,15079.68,2.67,3.0,2.33,3.25,3.67
1,1,Pingo Doce,2020,Madeira,Full-time,Early Career,Human Resources,Petroleum Engineer,1.0,4.0,...,Other,1,0,0,24315.54,3.00,2.5,2.67,2.50,2.33
2,2,Porto Editora,2020,Porto,Contract,Executive,Other,Engineering Manager,3.0,2.0,...,Non-Binary,0,0,0,14562.91,2.33,2.0,3.67,2.50,2.67
3,3,Tabaqueira,2020,Vila Real,Temporary,Senior-Level,Customer Service,iOS Engineer,4.0,3.0,...,Non-Binary,1,0,0,24504.66,3.67,4.0,2.33,2.75,3.33
4,4,Sonae,2005,Bragança,Internship,Senior-Level,Business Intelligence,Firefighter,4.0,4.0,...,Male,0,1,2,23397.48,4.00,2.0,3.33,3.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Ironhack,2009,Aveiro,Other,Early Career,IT,Procurement Manager,3.0,2.0,...,Female,1,1,1,25160.23,2.67,2.5,3.67,3.00,3.00
996,996,TIMWE,2020,Bragança,Part-time,Senior-Level,Engineering,EMT,3.0,3.0,...,Male,0,1,0,4767.75,3.00,3.5,3.33,3.75,2.67
997,997,Impresa,2013,Lisboa,Temporary,Early Career,Human Resources,Software Architect,3.0,4.0,...,Male,0,1,2,23914.41,3.67,2.5,2.33,3.50,3.33
998,998,SAPO,2020,Guarda,Full-time,Senior-Level,Management,Billing Specialist,4.0,4.0,...,Male,1,1,1,10601.55,4.00,3.0,2.00,3.25,3.67


In [68]:
# creating smaller DF
ratings = df[["company_id", "company_name", "location", "opportunity", "family", "schedule_flex", "enrichment", "culture", 
              "overall_satisfaction"]]

ratings

,company_id,company_name,location,opportunity,family,schedule_flex,enrichment,culture,overall_satisfaction
0,0,Jerónimo Martins,Lisboa,2.67,3.0,2.33,3.25,3.67,2.98
1,1,Pingo Doce,Madeira,3.00,2.5,2.67,2.50,2.33,2.60
2,2,Porto Editora,Porto,2.33,2.0,3.67,2.50,2.67,2.63
3,3,Tabaqueira,Vila Real,3.67,4.0,2.33,2.75,3.33,3.22
4,4,Sonae,Bragança,4.00,2.0,3.33,3.00,4.00,3.27
...,...,...,...,...,...,...,...,...,...
995,995,Ironhack,Aveiro,2.67,2.5,3.67,3.00,3.00,2.97
996,996,TIMWE,Bragança,3.00,3.5,3.33,3.75,2.67,3.25
997,997,Impresa,Lisboa,3.67,2.5,2.33,3.50,3.33,3.07
998,998,SAPO,Guarda,4.00,3.0,2.00,3.25,3.67,3.18


In [15]:
# exporting db to json
df.to_json(r"base_de_dados.json", orient = "records")

In [69]:
# exporting ratings db to json
ratings.to_json(r"ratings.json", orient = "records")

In [72]:
# exporting ratings db to csv
ratings.to_csv(directory + r"ratings.csv")

In [81]:
# reading updated json file
updated_data = pd.read_json(r"ratings.json")

In [82]:
updated_data

,company_id,company_name,location,opportunity,family,schedule_flex,enrichment,culture,overall_satisfaction
0,0,Jerónimo Martins,Lisboa,2.67,3.0,2.33,3.25,3.67,2.98
1,1,Pingo Doce,Madeira,3.00,2.5,2.67,2.50,2.33,2.60
2,2,Porto Editora,Porto,2.33,2.0,3.67,2.50,2.67,2.63
3,3,Tabaqueira,Vila Real,3.67,4.0,2.33,2.75,3.33,3.22
4,4,Sonae,Bragança,4.00,2.0,3.33,3.00,4.00,3.27
...,...,...,...,...,...,...,...,...,...
995,995,Ironhack,Aveiro,2.67,2.5,3.67,3.00,3.00,2.97
996,996,TIMWE,Bragança,3.00,3.5,3.33,3.75,2.67,3.25
997,997,Impresa,Lisboa,3.67,2.5,2.33,3.50,3.33,3.07
998,998,SAPO,Guarda,4.00,3.0,2.00,3.25,3.67,3.18


In [83]:
"""
def update_panel_json(input_json, target_key, update_value):
    if type(input_json) is dict and input_json:
        for key in input_json:
            if key == target_key:
                input_json[key] = update_value
            update_panel_json(input_json[key], target_key, update_value)

    elif type(input_json) is list and input_json:
        for entity in input_json:
            update_panel_json(entity, target_key, update_value)
"""

'\ndef update_panel_json(input_json, target_key, update_value):\n    if type(input_json) is dict and input_json:\n        for key in input_json:\n            if key == target_key:\n                input_json[key] = update_value\n            update_panel_json(input_json[key], target_key, update_value)\n\n    elif type(input_json) is list and input_json:\n        for entity in input_json:\n            update_panel_json(entity, target_key, update_value)\n'